In [3]:
! pip install --quiet environs cyksuid toolz psycopg2-binary typing_json backoff pyyaml facebook_business pystache

You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [4]:
! pip install git+https://github.com/nandanrao/typedjson-python

Defaulting to user installation because normal site-packages is not writeable


  Cloning https://github.com/nandanrao/typedjson-python to /tmp/pip-req-build-g21ntop_


  Installing build dependencies ... -

 \

 |

...

 done


  Getting requirements to build wheel ... done


    Preparing wheel metadata ... done


  Created wheel for typedjson: filename=typedjson-0.10.3-py3-none-any.whl size=6814 sha256=6b9944fcce3daf81b39946c19410221489fac71595fad4f97f4e7190cb8d4437
  Stored in directory: /tmp/pip-ephem-wheel-cache-ex4b5if1/wheels/06/6a/7f/41b926394656af81bedb9a1c1c94fec449a1ba11fc7f58125b
Successfully built typedjson


You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
from typing import List
import json

from environs import Env
import pandas as pd

from adopt.malaria import get_confs, get_df, load_basics, window, get_db_conf
from adopt.marketing import dict_from_nested_type
from adopt.marketing import Marketing
from adopt.facebook.state import CampaignState, get_api
from adopt.marketing import AudienceConf, CampaignConf
from adopt.campaign_queries import get_campaigns_for_user, create_campaign_confs, create_campaign_for_user

from mena.strata import read_general_conf, parse_conf_list

env = Env()
env.read_env('.env-vlab-vlab', override=True)
db_conf = get_db_conf(env)

In [7]:

GENERAL_CONF = read_general_conf('outs/jordan-general.csv')
DISTRIBUTION_FILE = 'outs/jordan-distribution.csv'
CREATIVE_FILE = 'outs/jordan-creative.csv'
CAMPAIGN = "vaccination-jordan"

In [8]:
USER = "dpinzonhernandez@worldbank.org"
AD_ACCOUNT = '246148586918843'
AD_CAMPAIGN = GENERAL_CONF['campaign_name']
TEMPLATE_CAMPAIGN = GENERAL_CONF['template_campaign_name']
PAGE_ID = GENERAL_CONF['page_id']
RESPONDENT_AUDIENCE = f"vlab-embed-{CAMPAIGN}-respondents"
INITIAL_SHORTCODE = GENERAL_CONF['welcome_screen']
SURVEY_SHORTCODES = parse_conf_list(GENERAL_CONF['survey_shortcodes'])
FINISHED_QUESTION_REF = GENERAL_CONF['finished_question_ref']
EXTRA_METADATA = json.loads(GENERAL_CONF['extra_metadata'])
DISTRIBUTION_VARS = parse_conf_list(GENERAL_CONF['distribution_vars'])
COUNTRY_CODE = GENERAL_CONF['country_code']

In [10]:
CAMPAIGNID = next(c['id'] for c in get_campaigns_for_user(USER, db_conf) 
                  if c['name'] == CAMPAIGN)

userinfo, config, db_conf, state, m, confs = load_basics(CAMPAIGNID, env)


# CREATE CAMPAIGN

In [ ]:
from adopt.facebook.update import Instruction
from adopt.malaria import run_instructions

def create_campaign(name):
    params = {
        "name": name,
        "objective": "MESSAGES",
        "status": "PAUSED",
        "special_ad_categories": [],
    }

    return Instruction("campaign", "create", params)

# create_campaign_for_user(USER, CAMPAIGN, db_conf)
# run_instructions([create_campaign(AD_CAMPAIGN)], state)

In [ ]:
# cid = next(c for c in state.campaigns if c['name'] == AD_CAMPAIGN)['id']
# run_instructions([Instruction("campaign", "update", {"status": "PAUSED"}, cid)], state)

# BASIC CONF

In [ ]:
c = {'optimization_goal': 'REPLIES',
     'destination_type': 'MESSENGER',
     'adset_hours': 48,
     'budget': 1400000.0,
     'min_budget': 100.0,
     'opt_window': 2*24,
     'end_date': '2021-05-25',
     'proportional': True,
     'page_id': PAGE_ID,
     'instagram_id': None,
     'ad_account': AD_ACCOUNT,
     'ad_campaign': AD_CAMPAIGN}

config = CampaignConf(**c)

create_campaign_confs(CAMPAIGNID, "opt", [config._asdict()], db_conf)

# AUDIENCES

In [41]:
SURVEY_SHORTCODES

['vaccjoen', 'vaccjoar']

In [ ]:
from adopt.marketing import dict_from_nested_type
import typedjson
import json

audiences = [
    {
        "name": f"vlab-vacc-{COUNTRY_CODE}-education-low",
        "shortcodes": SURVEY_SHORTCODES,
        "subtype": "LOOKALIKE",
        "lookalike": {
            "name": f"vlab-vacc-{COUNTRY_CODE}-education-low-lookalike",
            "target": 1100,
            "spec": {
                "country": COUNTRY_CODE,
                "starting_ratio": 0.0,
                "ratio": 0.2
            }
        },
        "question_targeting": {"op": "or", 
                               "vars": [
                                   {"op": "equal", "vars": [
                                       {"type": "response", "value": "education"},
                                       {"type": "constant", "value": "A"}]}, 
                                   {"op": "equal", "vars": [
                                       {"type": "response", "value": "education"},
                                       {"type": "constant", "value": "B"}]}
                               ]}
    },
    {
        "name": f"vlab-vacc-{COUNTRY_CODE}-education-high",
        "shortcodes": SURVEY_SHORTCODES,
        "subtype": "LOOKALIKE",
        "lookalike": {
            "name": f"vlab-vacc-{COUNTRY_CODE}-education-high-lookalike",
            "target": 1100,
            "spec": {
                "country": COUNTRY_CODE,
                "starting_ratio": 0.0,
                "ratio": 0.2
            }
        },
        "question_targeting": {"op": "or", 
                               "vars": [
                                   {"op": "equal", "vars": [
                                       {"type": "response", "value": "education"},
                                       {"type": "constant", "value": "C"}]}, 
                                   {"op": "equal", "vars": [
                                       {"type": "response", "value": "education"},
                                       {"type": "constant", "value": "D"}]}
                               ]}
    },
    {
        "name": RESPONDENT_AUDIENCE,
        "shortcodes": [INITIAL_SHORTCODE],
        "subtype": "CUSTOM"
    },
]


audience_confs = [typedjson.decode(AudienceConf, c) for c in audiences]
confs = [dict_from_nested_type(a) for a in audience_confs]

create_campaign_confs(CAMPAIGNID, "audience", confs, db_conf)

# CREATIVES

In [12]:
from mena.strata import generate_creative_confs

images = {i['name']: i for i in state.account.get_ad_images(fields=['name', 'hash'])}
creative_confs, image_confs = generate_creative_confs(CREATIVE_FILE, INITIAL_SHORTCODE, images)

create_campaign_confs(CAMPAIGNID, "creative", creative_confs, db_conf)

# STRATA

In [13]:
from mena.strata import get_adsets, extraction_confs, hyphen_case
from itertools import product 
from mena.strata import format_group_product

template_state = CampaignState(userinfo.token, 
                               get_api(env, userinfo.token), 
                               AD_ACCOUNT, 
                               TEMPLATE_CAMPAIGN)

a, g, l = get_adsets(template_state, extraction_confs)

variables = [
    { "name": "age", "source": "facebook", "conf": a},
    { "name": "gender", "source": "facebook", "conf": g},
    { "name": "location", "source": "facebook", "conf": l},
    { "name": "education", "source": "survey", "conf": {
        "levels": [{"name": "A+B", 
                    "audiences": [f"vlab-vacc-{COUNTRY_CODE}-education-low-lookalike"],
                    "excluded_audiences": [f"vlab-vacc-{COUNTRY_CODE}-education-high-lookalike"], 
                    "question_targeting": {"op": "or", 
                       "vars": [
                           {"op": "equal", "vars": [
                               {"type": "response", "value": "education"},
                               {"type": "constant", "value": "A"}]}, 
                           {"op": "equal", "vars": [
                               {"type": "response", "value": "education"},
                               {"type": "constant", "value": "B"}]}
                       ]}
                    },
                   {"name": "C+D", 
                    "audiences": [], 
                    "excluded_audiences": [f"vlab-vacc-{COUNTRY_CODE}-education-low-lookalike"], 
                    "question_targeting": {"op": "or", 
                       "vars": [
                           {"op": "equal", "vars": [
                               {"type": "response", "value": "education"},
                               {"type": "constant", "value": "C"}]}, 
                           {"op": "equal", "vars": [
                               {"type": "response", "value": "education"},
                               {"type": "constant", "value": "D"}]}

                       ]}
                    }

                   ]}}
]


share_lookup = pd.read_csv(DISTRIBUTION_FILE, header=[0,1,2], index_col=[0])

share = share_lookup.T.reset_index().melt(id_vars=DISTRIBUTION_VARS,
                                          var_name='location', 
                                          value_name='percentage')

groups = product(*[[(v['name'], v['source'], l) for l in v['conf']['levels']] for v in variables])
groups = [format_group_product(g, share) for g in groups]

/home/jupyter/.local/lib/python3.9/site-packages/facebook_business/utils/api_utils.py:30: UserWarning: Value of optimization_goal is not be compatible. Expect OptimizationGoal; got <class 'str'>
  warnings.warn(message)
INFO:root:Loaded 18 adsets from campaign template_jordan


In [14]:
ALL_CREATIVES = [t['name'] for t in image_confs]

def make_stratum(id_, quota, c):
    return { 'id': id_,
             'metadata': {**c['metadata'], **EXTRA_METADATA},
             'facebook_targeting': c['facebook_targeting'], 
             'creatives': ALL_CREATIVES,
             'audiences': c['audiences'],
             'excluded_audiences': [*c['excluded_audiences'], RESPONDENT_AUDIENCE],
             'quota': float(quota),
             'shortcodes': SURVEY_SHORTCODES,
             'question_targeting': c['question_targeting']}

In [ ]:
from adopt.marketing import StratumConf, QuestionTargeting

import typedjson
strata = [make_stratum(*g) for g in groups]
strata_data = [dict_from_nested_type(typedjson.decode(StratumConf, c)) for c in strata]
create_campaign_confs(CAMPAIGNID, "stratum", strata_data, db_conf)

# TESTING

In [6]:
mal = load_basics(CAMPAIGNID, env)

In [7]:
%%time

from adopt.malaria import update_ads_for_campaign, update_audience_for_campaign
 
# instructions, report = update_ads_for_campaign(mal)
instructions, report = update_audience_for_campaign(mal)

CPU times: user 1min 31s, sys: 155 ms, total: 1min 31s
Wall time: 1min 45s


In [ ]:
from adopt.malaria import run_instructions
from adopt.facebook.update import Instruction, GraphUpdater

run_instructions(instructions, mal.state)

In [70]:
import pandas as pd
from adopt.campaign_queries import get_last_adopt_report

rdf = pd.DataFrame(get_last_adopt_report(CAMPAIGNID, "FACEBOOK_ADOPT", mal.db_conf)).T